In [2]:
import requests
import json
import time
import math
import numpy as np
from astropy import wcs
from astropy.io import fits
import csv
import sys
from array import array
import os



apik = "sjwxwgavzfpucqaq"
directory = 'C:\\Users\\daniel\\Downloads\\'
file = '10B-1348_det_imagej.fits'
ra = 186.044243327
dec = 8.83186961532
radius = 8
cutoff = 300

dirtofile = directory+file

R1 = requests.post('http://nova.astrometry.net/api/login', data={'request-json': json.dumps({"apikey": apik})})
print("LOGIN:")
print(R1.json(),"\n")



fitsfile = fits.open(dirtofile)
data = fitsfile[0].data
header = fitsfile[0].header
data = 100.*(65535./(data-1900.))**1.4
print("transformation completed")

filet = file[0:-5]+"_t"+file[-5:]
dirtofilet = directory+filet

fits.writeto(dirtofilet, data, header, overwrite=True)
print("transformation file saved")



# url request
# url = "http://apod.nasa.gov/apod/image/1206/ldn673s_block1123.jpg"
# R2 = requests.post('http://nova.astrometry.net/api/upload', data={'request-json': json.dumps({"session": R1.json()["session"], "url": url, "scale_units": "degwidth", "scale_lower": 0.5, "scale_upper": 1.0, "center_ra": 290, "center_dec": 11, "radius": 2.0})})

R2 = requests.post('http://nova.astrometry.net/api/upload', data={'request-json': json.dumps({"publicly_visible": "y", "allow_modifications": "d", "session": R1.json()["session"], "allow_commercial_use": "d", "crpix_center": True, "scale_units": "degwidth", "scale_lower": 3, "scale_upper": 8, "center_ra": ra, "center_dec": dec, "radius": radius})}, files={'file': open(dirtofilet, 'rb')})
print("\nSUBMISSION:")
print(R2.json(),"\n")



r3strs = 'http://nova.astrometry.net/api/submissions/'
r3strm = str(R2.json()["subid"])
R3 = requests.post(str(r3strs + r3strm))
sys.stdout.write("solving ")
counter = 0

while len(R3.json()["job_calibrations"])==0:
    if (counter>cutoff):
        print("\nSolution failed, taking over",cutoff,"seconds\nExiting program")
        sys.exit()
    time.sleep(1)
    counter += 1
    sys.stdout.write(".")
    R3 = requests.post(str(r3strs + r3strm))

time.sleep(2)
print("\n\nSOLVED:")
print(R3.json())



r4strs = 'http://nova.astrometry.net/api/jobs/'
r4strm = str(R3.json()["jobs"])[1:-1]
r4stre1 = '/objects_in_field/'
r4stre2 = '/calibration/'
R4 = requests.post(str(r4strs+r4strm+r4stre2))
rasol1 = R4.json()["ra"]
decsol1 = R4.json()["dec"]



r5str = 'http://nova.astrometry.net/new_fits_file/' + r4strm
R5 = requests.get(r5str, allow_redirects=True)
sol1file = filet[0:-5]+"_sol1"+filet[-5:]
dirtosol1file = directory+sol1file

filesol1 = open(str(dirtosol1file),'wb')
filesol1.write(R5.content)
filesol1.close()
print("\nfirst solution file saved")

print()
print()
print()



print("SECOND RUN")
filesol1for2 = open(str(dirtosol1file),'rb')
R2 = requests.post('http://nova.astrometry.net/api/upload', data={'request-json': json.dumps({"publicly_visible": "y", "allow_modifications": "d", "session": R1.json()["session"], "allow_commercial_use": "d", "crpix_center": True, "scale_units": "degwidth", "scale_lower": 3, "scale_upper": 8, "center_ra": rasol1, "center_dec": decsol1, "radius": radius})}, files={'file': filesol1for2})
filesol1for2.close()
print("\nSUBMISSION:")
print(R2.json(),"\n")



r3strm = str(R2.json()["subid"])
R3 = requests.post(str(r3strs + r3strm))
sys.stdout.write("solving ")
counter = 0

while len(R3.json()["job_calibrations"])==0:
    if (counter>cutoff):
        print("\nSolution failed, taking over",cutoff,"seconds\nExiting program")
        sys.exit()
    time.sleep(1)
    counter += 1
    sys.stdout.write(".")
    R3 = requests.post(str(r3strs + r3strm))

time.sleep(2)
print("\n\nSOLVED:")
print(R3.json())



r4strm = str(R3.json()["jobs"])[1:-1]
R4 = requests.post(str(r4strs+r4strm+r4stre2))
rasol2 = R4.json()["ra"]
decsol2 = R4.json()["dec"]
orisol2 = R4.json()["orientation"]
scalesol2 = R4.json()["pixscale"]
radiussol2 = R4.json()["radius"]
htsol2 = (header['NAXIS2']*scalesol2)/3600.
wdsol2 = (header['NAXIS1']*scalesol2)/3600.

print("\nMETADATA:")
print("RA-FITS:",str(rasol2),"deg")
print("DEC-FITS:",str(decsol2),"deg")
print("SCANHT:",str(htsol2),"deg")
print("SCANWD:",str(wdsol2),"deg")
print("SCANRAD:",str(radiussol2),"deg")
print("SCANSCAL:",str(scalesol2),"arcsec/px")
print("SCAN-ORI: Up is",str(orisol2),"degrees E of N")
print()
print(str(rasol2),"deg"+","+str(decsol2),"deg"+","+str(htsol2),"deg"+","+str(wdsol2),"deg"+","+str(radiussol2),"deg"+","+str(scalesol2),"arcsec/px"+","+"Up is",str(orisol2),"degrees E of N")



r5str = 'http://nova.astrometry.net/new_fits_file/' + r4strm
R5 = requests.get(r5str, allow_redirects=True)
sol2file = filet[0:-5]+"_sol2"+filet[-5:]
dirtosol2file = directory+sol2file

filesol2 = open(str(dirtosol2file),'wb')
filesol2.write(R5.content)
filesol2.close()
print("\nsecond solution file saved")
      
print("\nPROGRAM COMPLETE")

LOGIN:
{'status': 'success', 'message': 'authenticated user: babnigg@uchicago.edu', 'session': '2xn20bp035we9lu09yaw54qegm0faxtx'} 

transformation completed
transformation file saved

SUBMISSION:
{'status': 'success', 'subid': 6130637, 'hash': '5a13a81ed2fb3ad5c1e2acb6dd7d4c8dc8ee9b68'} 

solving ..........................

SOLVED:
{'user': 36622, 'processing_started': '2022-08-11 19:05:13.374773', 'processing_finished': '2022-08-11 19:05:19.717747', 'user_images': [6268388], 'images': [14722271], 'jobs': [6837889], 'job_calibrations': [[6837889, 4982667]]}

first solution file saved



SECOND RUN

SUBMISSION:
{'status': 'success', 'subid': 6130646, 'hash': '5153a4d449a95b854ac66042591e9199177ebf97'} 

solving ........................

SOLVED:
{'user': 36622, 'processing_started': '2022-08-11 19:08:22.560560', 'processing_finished': '2022-08-11 19:08:27.616828', 'user_images': [6268397], 'images': [14722291], 'jobs': [6837898], 'job_calibrations': [[6837898, 4982676]]}

METADATA:
RA-F